In [0]:
import numpy as np
import pandas as pd
url=pd.read_csv('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv',sep='\t')

In [0]:
type(url.index)

pandas.core.indexes.range.RangeIndex

In [0]:
url.head


<bound method NDFrame.head of       order_id  ...  item_price
0            1  ...      $2.39 
1            1  ...      $3.39 
2            1  ...      $3.39 
3            1  ...      $2.39 
4            2  ...     $16.98 
...        ...  ...         ...
4617      1833  ...     $11.75 
4618      1833  ...     $11.75 
4619      1834  ...     $11.25 
4620      1834  ...      $8.75 
4621      1834  ...      $8.75 

[4622 rows x 5 columns]>

In [0]:
url.count()


order_id              4622
quantity              4622
item_name             4622
choice_description    3376
item_price            4622
dtype: int64

In [0]:
len(url.columns)

5

In [0]:
data_coloum=url.head(5)


In [0]:
data_coloum

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


In [0]:
dataframe=url

In [0]:
dataframe

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",$11.75
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",$11.75
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",$11.25
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",$8.75


In [0]:
dataframe.item_name

0                Chips and Fresh Tomato Salsa
1                                        Izze
2                            Nantucket Nectar
3       Chips and Tomatillo-Green Chili Salsa
4                                Chicken Bowl
                        ...                  
4617                            Steak Burrito
4618                            Steak Burrito
4619                       Chicken Salad Bowl
4620                       Chicken Salad Bowl
4621                       Chicken Salad Bowl
Name: item_name, Length: 4622, dtype: object

In [0]:
groupby_df = (dataframe.groupby(['item_name'])['item_name'].agg(['count']).sort_values(by='count', ascending=False).reset_index().drop_duplicates('item_name', keep='first'))
groupby_df

,item_name,count
0,Chicken Bowl,726
1,Chicken Burrito,553
2,Chips and Guacamole,479
3,Steak Burrito,368
4,Canned Soft Drink,301
5,Steak Bowl,211
6,Chips,211
7,Bottled Water,162
8,Chicken Soft Tacos,115
9,Chicken Salad Bowl,110


In [0]:
total_items_ordered = dataframe['quantity'].sum()
display(total_items_ordered)

4972

In [0]:
most_ordered_item_description = (
    dataframe.groupby(['choice_description'])['quantity']
    .sum()
    .sort_values(ascending=False)
    .reset_index()
    .iloc[0]
)

display(most_ordered_item_description)

choice_description    [Diet Coke]
quantity                      159
Name: 0, dtype: object

In [0]:
total_items_ordered = dataframe['quantity'].sum()
display(total_items_ordered)

4972

In [0]:
dataframe['item_price'] = dataframe['item_price'].astype(str).str.replace('$', '').astype(float)

/root/.ipykernel/4247/command-591170685594628-3531754717:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe['item_price'] = dataframe['item_price'].astype(str).str.replace('$', '').astype(float)


In [0]:
display(dataframe['item_price'].dtype)

dtype('float64')

In [0]:
dataframe['item_price'] = dataframe['item_price'].astype(str).apply(lambda x: float(x.replace('$', '')))

In [0]:
display(dataframe['item_price'].dtype)

dtype('float64')

In [0]:
revenue = dataframe['item_price'].sum()
display(revenue)

34500.16

In [0]:
orders_count = dataframe['order_id'].nunique()
display(orders_count)

1834

In [0]:
from pyspark.sql.functions import sum, avg

# Convert the Pandas DataFrame to a Spark DataFrame
spark_df = spark.createDataFrame(dataframe)

# Calculate the average revenue per order
avg_revenue_per_order = spark_df.groupBy("order_id").agg(sum("item_price").alias("total_revenue")).agg(avg("total_revenue"))

display(avg_revenue_per_order)

avg(total_revenue)
18.811428571428692


In [0]:
dataframe

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39
1,1,1,Izze,[Clementine],3.39
2,1,1,Nantucket Nectar,[Apple],3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98
...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",11.75
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",11.75
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",11.25
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",8.75


In [0]:
distinct_items_count = spark_df.select("item_name").distinct().count()
display(distinct_items_count)

50

In [0]:
%pip install pyspark

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

# Load the data
# Load the data using Delta format instead of CSV
# Load the data using CSV format instead of Delta
data = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("https://raw.githubusercontent.com/deepanshuMeteor/ComputerVision/refs/heads/main/data.csv")

# Prepare the data for machine learning
feature_columns = data.columns[:-1]  # Assuming the last column is the target variable
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data_prepared = assembler.transform(data)

# Split the data into training and test sets
train_data, test_data = data_prepared.randomSplit([0.7, 0.3])

# Define the model
lr = LinearRegression(featuresCol="features", labelCol=data.columns[-1])

# Create a pipeline
pipeline = Pipeline(stages=[assembler, lr])

# Train the model
model = pipeline.fit(train_data)

# Make predictions
predictions = model.transform(test_data)

# Display the predictions
display(predictions.select("prediction", data.columns[-1]))

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-591170685594645>, line 13
      5 from pyspark.ml import Pipeline
      7 # Load the data
      8 # Load the data using Delta format instead of CSV
      9 # Load the data using CSV format instead of Delta
     10 data = spark.read.format("csv") \
     11     .option("header", "true") \
     12     .option("inferSchema", "true") \
---> 13     .load("https://raw.githubusercontent.com/deepanshuMeteor/ComputerVision/refs/heads/main/data.csv")
     15 # Prepare the data for machine learning
     16 feature_columns = data.columns[:-1]  # Assuming the last column is the target variable

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     4

In [0]:
%restart_python

In [0]:
dbutils.library.restartPython()

In [0]:
%pip install pyspark

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

# Load the data
data = spark.read.format("delta") \
    .load("https://raw.githubusercontent.com/deepanshuMeteor/ComputerVision/refs/heads/main/data.csv")

# Prepare the data for machine learning
feature_columns = data.columns[:-1]  # Assuming the last column is the target variable
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data_prepared = assembler.transform(data)

# Split the data into training and test sets
train_data, test_data = data_prepared.randomSplit([0.7, 0.3])

# Define the model
lr = LinearRegression(featuresCol="features", labelCol=data.columns[-1])

# Create a pipeline
pipeline = Pipeline(stages=[assembler, lr])

# Train the model
model = pipeline.fit(train_data)

# Make predictions
predictions = model.transform(test_data)

# Display the predictions
display(predictions.select("prediction", data.columns[-1]))

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


---------------------------------------------------------------------------
UnsupportedOperationException             Traceback (most recent call last)
File <command-591170685594667>, line 9
      5 from pyspark.ml import Pipeline
      7 # Load the data
      8 data = spark.read.format("delta") \
----> 9     .load("https://raw.githubusercontent.com/deepanshuMeteor/ComputerVision/refs/heads/main/data.csv")
     11 # Prepare the data for machine learning
     12 feature_columns = data.columns[:-1]  # Assuming the last column is the target variable

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:312, in DataFrame